In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/endsem-eval-nlp/val.arrow
/kaggle/input/endsem-eval-nlp/test.arrow
/kaggle/input/endsem-eval-nlp/train.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/val/cache-ed401026e9103c65.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/val/state.json
/kaggle/input/next-utterance-prediction-nlp-2025/val/dataset_info.json
/kaggle/input/next-utterance-prediction-nlp-2025/val/data-00000-of-00001.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/val/cache-cc3928cf36c83499.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/val/cache-00884308fd07a1b2.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/test/state.json
/kaggle/input/next-utterance-prediction-nlp-2025/test/dataset_info.json
/kaggle/input/next-utterance-prediction-nlp-2025/test/data-00000-of-00001.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/train/state.json
/kaggle/input/next-utterance-prediction-nlp-2025/train/dataset_info.json
/kaggle/input/next-utterance-prediction-nlp-2025/train/cache-

In [7]:
from datasets import Dataset 
from datasets import load_from_disk

# Load from arrow file directly
train = Dataset.from_file("/kaggle/input/endsem-eval-nlp/val.arrow")
test = Dataset.from_file("/kaggle/input/endsem-eval-nlp/test.arrow")
val = Dataset.from_file("/kaggle/input/endsem-eval-nlp/train.arrow")

In [8]:
print(train)
print(val)
print(test)
print(type(train))

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 4008
})
Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 576
})
Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 968
})
<class 'datasets.arrow_dataset.Dataset'>


In [9]:
train_df = train.to_pandas()
test_df= test.to_pandas()
val_df = val.to_pandas()

In [10]:
train_df.head(4)

,input_text,target_text
0,T: Hi you how to do it today? [SEP] P: Great. ...,I'm doing well. Thanks for asking.
1,T: Hi you how to do it today? [SEP] P: Great. ...,So you're doing great.
2,T: Hi you how to do it today? [SEP] P: Great. ...,I know your brother brought you in today and h...
3,T: Hi you how to do it today? [SEP] P: Great. ...,"Alright, so you feel like, everything's kind o..."


In [11]:
train_df.iloc[0, 0]

'T: Hi you how to do it today? [SEP] P: Great. How are you?'

In [12]:
train_df.iloc[0 ,  0]

'T: Hi you how to do it today? [SEP] P: Great. How are you?'

In [13]:
train_df.iloc[3,  0]

"T: Hi you how to do it today? [SEP] P: Great. How are you? [SEP] T: I'm doing well. Thanks for asking. [SEP] T: So you're doing great. [SEP] P: I'm doing awesome. [SEP] T: I know your brother brought you in today and he had expressed some concerns about your mood. Do you know what that's about? [SEP] P: I, I think he's worrying for no reason. Because I, you know, I for the last like week and a half, I have been following my passion, which is baking. And I didn't realize this before, because I worked in an office. And I just realized that that's not what I want to do with my life. And I know what I want to do with my life now, and I've never known before. So this is an amazing feeling. And what I want to do is I want to start my own baking business. And so what happened was That a couple. Two weeks ago, my nephew had a bake sale. And I've always I've always been like baking, but I've always been like from the box or like from, you know, whatever. So I tried for the first time making br

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")
tokenizer.pad_token = tokenizer.eos_token


In [15]:
def preprocess(example):
    full_text = example['input_text'] + tokenizer.eos_token + example['target_text'] + tokenizer.eos_token
    tokenized = tokenizer(full_text, truncation=True, padding='max_length', max_length=512)
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'labels': tokenized['input_ids']  # For language modeling
    }

### We are adding both `input_ids` and `labels` in the output of the preprocessing function, even though they have the same values (i.e., ``tokenized['input_ids'])``. This is because we are doing a next-token prediction task (language modeling). 

### In preprocessing, we are combining the input (like a prompt or training part) and the target (like the expected output or test part) into a single sequence. Then, we tokenize this combined sequence. Both input_ids and labels are set to this tokenized sequence. Later during training, the model will use input_ids as input and labels to calculate the loss by predicting the next tokens in the sequence.

In [16]:
processed = [preprocess(row) for _, row in train_df.iterrows()]
val_processed = [preprocess(row) for _, row in val_df.iterrows()]
test_processed = [preprocess(row) for _, row in test_df.iterrows()]
tokenized_dataset = Dataset.from_list(processed)
tokenized_val_dataset =  Dataset.from_list(val_processed)
tokenized_test_dataset = Dataset.from_list(test_processed)

In [17]:
print(processed[0])
print(len(tokenized_dataset))
print(model.config)
import os
os.environ["WANDB_DISABLED"] = "true"


{'input_ids': [51, 25, 15902, 345, 703, 284, 466, 340, 1909, 30, 685, 5188, 47, 60, 350, 25, 3878, 13, 1374, 389, 345, 30, 50256, 40, 1101, 1804, 880, 13, 6930, 329, 4737, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results-small",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    logging_dir="./logs-small",
    eval_strategy="epoch",  # no evaluation during training
    save_strategy="no",        # no checkpoints saved
    report_to="none"           # disable reporting to tools like WandB
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # use the full dataset now
    eval_dataset=tokenized_val_dataset,
)

trainer.train()


In [18]:
## saving the model 
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [19]:
def generate_response(input_text):
    # Move model to GPU
    model.to('cuda')  # Ensure the model is on the GPU

    # Tokenize input text
    input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')

    # Move input_ids to GPU
    input_ids = input_ids.to('cuda')  # Move input_ids to the same device as the model

    # Generate response from the model
    output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.9, temperature=0.8)

    # Decode the response
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Filter out any unwanted tokens such as [SEP] or 'P:'
    response = response.replace('[SEP]', '').replace('P:', '').strip()

    return response

# Test with a custom sentence entered by the user
user_input = "Hello, i am sick and tired "  # Example, change this input to test with different sentences
response = generate_response(user_input)
print(f"Response: {response}")


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpec

Response: Hello, i am sick and tired Hi, I'm dad.


In [20]:
eval_results = trainer.evaluate(tokenized_test_dataset)

# Print evaluation results
print("Evaluation Results on Test Set:")
print(eval_results)

NameError: name 'trainer' is not defined

In [61]:
pip install evaluate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [30]:
import evaluate
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

def evaluate_model(test_dataset):
    predictions = []
    references = []

    for example in test_dataset:
        input_text = tokenizer.decode(example['input_ids'], skip_special_tokens=True)

        inputs = tokenizer(input_text + tokenizer.eos_token, return_tensors='pt', padding=True).to('cuda')

        output = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=True,
            no_repeat_ngram_size=2,
            top_k=50,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        predictions.append(generated_text)

        # ✅ FIXED: decode label ids to text
        reference = tokenizer.decode(example['labels'], skip_special_tokens=True)
        references.append(reference)

    # Compute BLEU — needs tokenized references
    bleu_results = bleu.compute(
        predictions=predictions,
        references=[[ref.split()] for ref in references]
    )

    # Compute BERTScore — raw string references
    bertscore_results = bertscore.compute(
        predictions=predictions,
        references=references,
        lang='en'
    )

    print(f"BLEU Score: {bleu_results}")
    print(f"BERTScore: {bertscore_results}")

# Run evaluation
evaluate_model(tokenized_test_dataset)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU Score: {'bleu': 0.0023545172006527518, 'precisions': [0.3066479553539375, 0.07605098586941476, 0.03749019291266079, 2.5288989932453106e-06], 'brevity_penalty': 0.34336288348406097, 'length_ratio': 0.48333288842402605, 'translation_length': 398333, 'reference_length': 824138}
BERTScore: {'precision': [0.9513427019119263, 0.9946163892745972, 0.9984627962112427, 0.9999999403953552, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999403953552, 0.9987034797668457, 0.9999999403953552, 0.9999999403953552, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9375048875808716, 0.9834198355674744, 0.9832203984260559, 0.9999998807907104, 0.9954245090484619, 0.9979285001754761, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9810988903045654, 0.9678171277046204, 0.9790961742401123, 0.9999999403953552, 0.9964114427566528, 0.9999998807907104, 0.996764063835144, 0.9987472295761108, 0.9504750967025757, 0.96

In [27]:
pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [25]:
import evaluate